# One Shot

Ziel ist es, mit einem Request alle Antworten zu erhalten.

In [20]:
import sqlite3
import response_format
from dotenv import load_dotenv
from litellm import completion

load_dotenv()

connection = sqlite3.connect('../data/survey.db')
cursor = connection.cursor()

In [21]:
# strategie laden
cursor.execute("SELECT id, system_path, message_path FROM strategies WHERE name = 'oneshot_none'")
strategy_id, system_path, message_path = cursor.fetchone()

# prompts lesen
with open(f"../{system_path}", "r") as f:
    system_prompt = f.read()

with open(f"../{message_path}", "r") as f:
    message_prompt = f.read()

In [22]:
# fragen laden
cursor.execute("SELECT id, text FROM questions")
questions = cursor.fetchall()

# prompt anreichern
questions_text = "\n".join([f"{id}. {text}" for id, text in questions])
content = message_prompt.replace("{questions}", questions_text)

In [23]:
# models laden
models_to_test = [
    'Mistral Small',
    'Mistral Large',
    # 'GPT-5.2 Nano',
    # 'GPT-5.2',
    # 'Claude Haikaku',
    # 'Claude Opus',
    'Gemini Flash',
    # 'Gemini Pro',
    # 'Deepseek Chat',
    # 'Deepseek Reasoner'
]

placeholder = ','.join(['?' for _ in models_to_test])

cursor.execute(
    f"SELECT id, model_id FROM models WHERE name IN ({placeholder})",
    models_to_test
)

models = cursor.fetchall()

In [24]:
responses = []

for model_db_id, model_id in models:

    response = completion(
        model = model_id,
        messages = [
            {"role": "user", "content": content}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "survey_response",
                "schema": response_format.CompleteResponseFormat.model_json_schema()
            }
        }
    )

    responses.append((model_db_id, response))

In [25]:
# response in die datenbank schreiben

for model_db_id, response in responses:

    parsed_response = response_format.CompleteResponseFormat.model_validate_json(response.choices[0].message.content)

    for item in parsed_response.answers:
        cursor.execute(
            "INSERT INTO responses (model_id, strategy_id, question_id, answer) VALUES (?,?,?,?)",
            (model_db_id, strategy_id, item.question, item.answer)
        )

connection.commit()

In [26]:
# simple auswertung